In [2]:
import wrds
import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### INPUTS - 3 years of data will take about 2 hours to run and output database size will be ~ 2.5GB

In [3]:
path_tickers = r'Complete-List-of-Biotech-Stocks-Listed-on-NASDAQ-Jan-1-24.xlsx'  #[Dec 2, Shravan] joining file name with directory as well
tickers_df = pd.read_excel(path_tickers)
# tickers_df = pd.read_excel(r"Complete-List-of-Biotech-Stocks-Listed-on-NASDAQ-Jan-1-24.xlsx")
tickers = tickers_df['Ticker'].unique().tolist()

# Define the dates
start_year = 2013
start_month = 1
start_day = 1

end_year = 2023
end_month = 12
end_day = 30

# Create start_date and end_date using datetime
start_date = datetime(start_year, start_month, start_day)
end_date = datetime(end_year, end_month, end_day)

# Generate list of years between the start and end date (inclusive)
years = list(range(start_date.year, end_date.year + 1))

# Print the start and end dates
print("Start Date:", start_date.strftime("%Y-%m-%d"))
print("End Date:", end_date.strftime("%Y-%m-%d"))

stocks = tickers_df['Ticker']


Start Date: 2013-01-01
End Date: 2023-12-30


### EQUITIES DATA

In [4]:
# # Function to get CUSIPs for tickers
def get_cusips(tickers):
    # Create a string of tickers in the form of ('ticker1', 'ticker2', ...)
    tickers_str = "('" + "','".join(tickers) + "')"

    query = f"""
        SELECT
             c.htsymbol, c.hcusip
        FROM
            crsp_a_stock.dsfhdr AS c
        WHERE
            c.htsymbol IN {tickers_str}
    """

    # Execute the query to get header data and return CUSIPs
    header = conn.raw_sql(query)

    # Remove rows with missing CUSIPs
    cusips = header.dropna()['hcusip'].unique().tolist()  # Ensure unique and drop NaN values
    return cusips

def get_permnos(tickers):
    stock_str = "('" + "','".join(stocks)+ "')"
    query = f"""
        SELECT
             *
        FROM
            crsp_a_stock.dsfhdr AS c
        WHERE
            c.htsymbol IN {stock_str}
    """

    header = conn.raw_sql(query)
    return header.dropna()['permno']
permnos = get_permnos(stocks)

def query_daily_data(start_day: str, end_day: str, permnos: list):
    permnos_str = "(" + ",".join([str(i) for i in permnos])+ ")"

    query = f"""
    SELECT * FROM (
            SELECT
                c.permno,
                c.cusip,
                c.date,
                c.bid,
                c.ask,
                c.vol,
                c.shrout, -- shares outstanding
                c.prc, -- price
               (c.prc * c.shrout) AS mktcap -- calculated market cap
            FROM
                crsp_a_stock.dsf AS c
            WHERE
                c.date BETWEEN '{start_day}' AND '{end_day}'
                AND c.permno IN {permnos_str}
            ) as subquery
        WHERE
            subquery.mktcap BETWEEN 100000 AND 5000000000
    """

    data = conn.raw_sql(query)

    return data

In [5]:
data = query_daily_data(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"), permnos)
equities_data = data

: 

### OPTIONS DATA

In [ ]:
def get_tickers_to_permno_mapping(tickers):
    # Convert 'begdat' and 'enddat' to datetime
    tickers_str = "('" + "','".join(tickers) + "')"
    query = f"""
            SELECT *
            FROM
                crsp_a_stock.dsfhdr AS c
            WHERE
                c.htsymbol IN {tickers_str}
        """

    df = conn.raw_sql(query)
    df['begdat'] = pd.to_datetime(df['begdat'])
    df['enddat'] = pd.to_datetime(df['enddat'])

    # Initialize the mapping dictionary
    ticker_permno_mapping = {}

    # Iterate over the rows to create the desired dictionary
    for _, row in df.iterrows():
        ticker = row['htsymbol']
        permno = row['permno']
        date_tuple = (row['begdat'], row['enddat'])

        # Check if the ticker is already in the mapping
        if ticker not in ticker_permno_mapping:
            ticker_permno_mapping[ticker] = {}

        # Add the date tuple and corresponding permno to the ticker's dictionary
        ticker_permno_mapping[ticker][date_tuple] = permno

    return ticker_permno_mapping

def fetch_options_data(tickers, start_date, end_date, years):
    """
    Fetch options data for given tickers across multiple years with improved error handling
    and data validation.
    """
    all_options_data = []  # List to store concatenated results
    valid_data_found = False  # Flag to track if any valid data was found

    # Input validation
    if not tickers or not isinstance(tickers, list): #checking for not null or list instance
        raise ValueError("Tickers must be provided as a non-empty list")
    if not years or not isinstance(years, list): #checking for not null or list instance
        raise ValueError("Years must be provided as a non-empty list")

    # Loop over each year in the list
    for year in years:
        year_data = None
        try:
            table = 'opprcd' + str(year)
            # Create the SQL query with multiple tickers using IN clause
            tickers_str = "','".join(tickers)
            query = f"""
                SELECT o.date, s.ticker, o.exdate, o.cp_flag, o.strike_price,
                       o.strike_price / 10000 AS strike_price_actual,
                       o.best_bid, o.best_offer, o.volume, o.open_interest,
                       o.impl_volatility, o.delta, o.gamma, o.vega, o.theta
                FROM optionm.{table} o
                JOIN (
                    SELECT DISTINCT secid, ticker
                    FROM optionm.secnmd
                ) AS s ON o.secid = s.secid
                WHERE s.ticker IN ('{tickers_str}')
                  AND o.date BETWEEN '{start_date}' AND '{end_date}'
                ORDER BY o.date, s.ticker, o.exdate, o.strike_price;
            """

            # Execute the query
            year_data = conn.raw_sql(query)
            # ipdb.set_trace(context=5)

            # Validate returned data
            if year_data is not None and isinstance(year_data, (list, pd.DataFrame)):
                if len(year_data) > 0:
                    # Check if data has the expected structure
                    expected_columns = ['date', 'ticker', 'exdate', 'cp_flag', 'strike_price',
                                     'best_bid', 'best_offer', 'volume', 'open_interest',
                                     'impl_volatility', 'delta', 'gamma', 'vega', 'theta']

                    if isinstance(year_data, pd.DataFrame):
                        if all(col in year_data.columns for col in expected_columns):
                            all_options_data.append(year_data)
                            valid_data_found = True
                            print(f"\nOptions data for year {year}:")
                            print(f"Total number of options contracts: {len(year_data)}")
                            print("\nSample of options data:")
                            print(year_data.head())  # Display first 5 records
                        else:
                            print(f"Warning: Data for year {year} has unexpected structure")
                    else:  # If it's a list
                        all_options_data.extend(year_data)
                        valid_data_found = True
                        print(f"\nOptions data for year {year}:")
                        print(f"Total number of options contracts: {len(year_data)}")
                        print("\nSample of options data:")
                        print(year_data[:5])  # Display first 5 records
                else:
                    print(f"No data found for year {year}")
            else:
                print(f"Invalid or empty data returned for year {year}")

        except pd.errors.EmptyDataError:
            print(f"No data available for year {year}")
        except (ValueError, AttributeError) as e:
            print(f"Data validation error for year {year}: {e}")
        except Exception as e:
            if "UndefinedTable" in str(e):
                print(f"Table for year {year} does not exist in the database")
            else:
                print(f"Error processing data for year {year}: {e}")

    # Final validation of concatenated data
    if not valid_data_found:
        print("\nWarning: No valid data was found for any year")
        return pd.DataFrame()  # Return empty DataFrame instead of None

    # Combine all data
    if all_options_data:
        if isinstance(all_options_data[0], pd.DataFrame):
            final_data = pd.concat(all_options_data, ignore_index=True)
        else:
            final_data = pd.DataFrame(all_options_data)

        # Display the overall summary
        print("\nOverall options data summary:")
        print(f"Total number of options contracts fetched: {len(final_data)}")
        return final_data
    else:
        return pd.DataFrame()  # Return empty DataFrame if no data

# Usage example:
try:
    options_data = fetch_options_data(tickers, start_date, end_date, years)
    if not options_data.empty:
        print("\nData fetched successfully")
    else:
        print("\nNo valid data was returned")
except Exception as e:
    print(f"Error in main execution: {e}")


Options data for year 2013:
Total number of options contracts: 3484188

Sample of options data:
         date ticker      exdate cp_flag  strike_price  strike_price_actual  \
0  2013-01-02   ACAD  2013-01-19       C         500.0                 0.05   
1  2013-01-02   ACAD  2013-01-19       P         500.0                 0.05   
2  2013-01-02   ACAD  2013-01-19       C        1000.0                 0.10   
3  2013-01-02   ACAD  2013-01-19       P        1000.0                 0.10   
4  2013-01-02   ACAD  2013-01-19       C        1500.0                 0.15   

   best_bid  best_offer  volume  open_interest  impl_volatility     delta  \
0       4.1        4.40     0.0            0.0         6.179519  0.989860   
1       0.0        0.05     0.0            0.0         5.835941 -0.007923   
2       3.6        3.90     0.0           20.0         4.239125  0.984304   
3       0.0        0.05     0.0           22.0         3.989646 -0.012833   
4       3.1        3.40     0.0           1

### 80% Liquidity Constraint on Options Data

In [6]:
volume_threshold_options_data = options_data[options_data['volume'] > np.percentile(options_data['volume'], 80)]
print(volume_threshold_options_data.shape)
print(options_data.shape)
ticker_to_permno_mapping = get_tickers_to_permno_mapping(tickers)

(8199997, 15)
(55799127, 15)


In [7]:
def get_permno(row, ticker_to_permno_mapping, asofdate):
    '''function to get permno corresponding to ticker asofdate'''
    ticker = row['ticker']
    for date_tuple, permno in ticker_to_permno_mapping.get(ticker, {}).items():
        if date_tuple[0] <= asofdate <= date_tuple[1]:
            return permno
    return None  # If today's date is not in any of the date ranges

# Apply the function to create the 'permno' column
volume_threshold_options_data['permno'] = volume_threshold_options_data.apply(get_permno, axis=1, args=(ticker_to_permno_mapping, pd.Timestamp('2023-12-01'),))

print(volume_threshold_options_data.shape)
print(volume_threshold_options_data['permno'].isna().any())

(8199997, 16)
False


/var/folders/1w/11tc25ps6ts6s9qzm42mm3j40000gn/T/ipykernel_17756/3719623979.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_threshold_options_data['permno'] = volume_threshold_options_data.apply(get_permno, axis=1, args=(ticker_to_permno_mapping, pd.Timestamp('2023-12-01'),))


In [8]:
equities_data.head()

,permno,cusip,date,bid,ask,vol,shrout,prc,mktcap
0,10200,75991610,2020-01-02,93.72,93.78,313953.0,52078.0,93.780,4883874.84
1,10200,75991610,2020-01-03,92.74,92.78,246742.0,52078.0,92.740,4829713.72
2,10200,75991610,2020-01-06,93.89,93.90,380649.0,52078.0,93.895,4889863.81
3,10200,75991610,2020-01-07,95.00,95.08,370624.0,52078.0,95.000,4947410.00
4,10200,75991610,2020-01-08,95.13,95.14,259393.0,52078.0,95.130,4954180.14


In [9]:
volume_threshold_options_data.head()

,date,ticker,exdate,cp_flag,strike_price,strike_price_actual,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,permno
0,2020-01-02,ABEO,2020-01-17,C,2500.0,0.25,0.50,1.00,87.0,1661.0,1.076193,0.895771,0.258311,0.117540,-1.576614,87656
1,2020-01-02,ABEO,2020-01-17,P,2500.0,0.25,0.05,0.15,16.0,760.0,1.509266,-0.165591,0.253322,0.161895,-2.962448,87656
3,2020-01-02,ABEO,2020-01-17,C,5000.0,0.50,0.05,0.10,40.0,2994.0,1.773509,0.146708,0.199077,0.149495,-3.232589,87656
15,2020-01-02,ABEO,2020-01-17,P,20000.0,2.00,16.60,17.20,1.0,1.0,5.122027,-0.894074,0.055954,0.118863,-7.235096,87656
22,2020-01-02,ABEO,2020-02-21,P,2500.0,0.25,0.05,0.30,11.0,33.0,1.067041,-0.201668,0.222041,0.334315,-1.287748,87656


### MERGED DATA

In [10]:
# Convert date columns to datetime type for accurate merging
equities_data['date'] = pd.to_datetime(equities_data['date'])
volume_threshold_options_data['date'] = pd.to_datetime(volume_threshold_options_data['date'])

# Merging both datasets on 'ticker' and 'date'
merged_data = pd.merge(
    equities_data.drop_duplicates(),
    volume_threshold_options_data,
    on=['permno', 'date'],
    how='inner'  # 'inner' merge to keep only rows that have a match in both datasets
)

# Display the merged data
merged_data.head()

/var/folders/1w/11tc25ps6ts6s9qzm42mm3j40000gn/T/ipykernel_17756/1432589667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_threshold_options_data['date'] = pd.to_datetime(volume_threshold_options_data['date'])


,permno,cusip,date,bid,ask,vol,shrout,prc,mktcap,ticker,...,strike_price_actual,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta
0,10200,75991610,2020-01-02,93.72,93.78,313953.0,52078.0,93.78,4883874.84,RGEN,...,9.5,1.40,1.75,2.0,98.0,0.310298,0.396742,0.067700,7.253972,-22.242290
1,10200,75991610,2020-01-02,93.72,93.78,313953.0,52078.0,93.78,4883874.84,RGEN,...,7.5,0.25,0.40,10.0,528.0,0.390420,-0.053629,0.008045,3.776815,-5.439961
2,10200,75991610,2020-01-02,93.72,93.78,313953.0,52078.0,93.78,4883874.84,RGEN,...,8.0,13.00,14.80,2.0,377.0,0.270031,0.953724,0.012104,3.225182,-2.681417
3,10200,75991610,2020-01-02,93.72,93.78,313953.0,52078.0,93.78,4883874.84,RGEN,...,8.5,8.40,11.70,1.0,244.0,0.337758,0.799340,0.024216,9.627598,-11.171340
4,10200,75991610,2020-01-02,93.72,93.78,313953.0,52078.0,93.78,4883874.84,RGEN,...,9.0,6.10,7.30,11.0,119.0,0.343478,0.646064,0.031282,12.834230,-15.464910


### Output to sqlite database so that it can handle files of all sizes

In [11]:
# Step 1: Create or connect to the SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('1_financial_data.db')  # Creates the database file 'financial_data.db' if it doesn't exist

# Step 2: Write DataFrames to the SQLite database

# Write equities_data to a table called 'equities_data'
equities_data.to_sql('equities_data', conn, if_exists='replace', index=False)

# Write volume_threshold_options_data to a table called 'volume_threshold_options_data'
volume_threshold_options_data.to_sql('volume_threshold_options_data', conn, if_exists='replace', index=False)

# Write merged_data to a table called 'merged_data'
merged_data.to_sql('merged_data', conn, if_exists='replace', index=False)

# Step 3: Commit changes and close the connection
conn.commit()
conn.close()

print("Data has been written to the SQLite database successfully!")

Data has been written to the SQLite database successfully!


### Final check for duplicates

In [12]:
has_duplicates = merged_data.duplicated().any()

if has_duplicates:
    print("The DataFrame contains duplicates.")
else:
    print("The DataFrame does not contain any duplicates.")

The DataFrame does not contain any duplicates.
